In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import os
import sys

import tensorflow as tf

import yfinance as yf

2025-10-09 12:40:03.357728: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-10-09 12:40:03.358200: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-09 12:40:03.439995: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-10-09 12:40:05.880933: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To tur

# Important!
#  Data is from 1980-12-12 to 2025-10-03

# Loading the data

Analysis and holdings

In [2]:
earnings_estimate = pd.read_csv('data/analysis_and_holdings/AAPL_earnings_estimate.csv')
earnings_history = pd.read_csv('data/analysis_and_holdings/AAPL_earnings_history.csv')
eps_revisions = pd.read_csv('data/analysis_and_holdings/AAPL_eps_revisions.csv')
eps_trend = pd.read_csv('data/analysis_and_holdings/AAPL_eps_trend.csv')
growth_estimates = pd.read_csv('data/analysis_and_holdings/AAPL_growth_estimates.csv')
insider_purchases = pd.read_csv('data/analysis_and_holdings/AAPL_insider_purchases.csv')
insider_roster_holders = pd.read_csv('data/analysis_and_holdings/AAPL_insider_roster_holders.csv')
institutional_holders = pd.read_csv('data/analysis_and_holdings/AAPL_institutional_holders.csv')
major_holders = pd.read_csv('data/analysis_and_holdings/AAPL_major_holders.csv')
mutualfunds_holders = pd.read_csv('data/analysis_and_holdings/AAPL_mutualfund_holders.csv')
reccomendations = pd.read_csv('data/analysis_and_holdings/AAPL_recommendations.csv')
revenue_estimate = pd.read_csv('data/analysis_and_holdings/AAPL_revenue_estimate.csv')
sustainability = pd.read_csv('data/analysis_and_holdings/AAPL_sustainability.csv')
upgrades_downgrades = pd.read_csv('data/analysis_and_holdings/AAPL_upgrades_downgrades.csv')

Financials

In [3]:
balance_sheet = pd.read_csv('data/financials/AAPL_balance_sheet.csv', index_col=0)
cashflow = pd.read_csv('data/financials/AAPL_cashflow.csv', index_col=0)
income_statement = pd.read_csv('data/financials/AAPL_income_statement.csv', index_col=0)

Info

In [4]:
actions = pd.read_csv('data/info/AAPL_actions.csv', index_col=0)
capital_gains = pd.read_csv('data/info/AAPL_capital_gains.csv', index_col=0)
dividends = pd.read_csv('data/info/AAPL_dividends.csv', index_col=0)
fast_info = pd.read_csv('data/info/AAPL_fast_info.csv', index_col=0)
history = pd.read_csv('data/info/AAPL_history.csv', index_col=0)
info = pd.read_csv('data/info/AAPL_info.csv')
splits = pd.read_csv('data/info/AAPL_splits.csv', index_col=0)

Price

In [5]:
ohlc = pd.read_csv('data/price/AAPL_price_volume.csv', index_col=0, skiprows=2)
vix = pd.read_csv('../VIX/data/price/VIX_price_volume.csv', header=None, skiprows=3)

# Data exploration

Price

In [6]:
ohlc.tail()

,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5
Date,,,,,
2025-09-29,254.429993,255.000000,253.009995,254.559998,40127700
2025-09-30,254.630005,255.919998,253.110001,254.860001,37704300
2025-10-01,255.449997,258.790009,254.929993,255.039993,48713900
2025-10-02,257.130005,258.179993,254.149994,256.579987,42630200
2025-10-03,258.019989,259.239990,253.949997,254.669998,49107000


In [7]:
ohlc.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11294 entries, 1980-12-12 to 2025-10-03
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 1  11294 non-null  float64
 1   Unnamed: 2  11294 non-null  float64
 2   Unnamed: 3  11294 non-null  float64
 3   Unnamed: 4  11294 non-null  float64
 4   Unnamed: 5  11294 non-null  int64  
dtypes: float64(4), int64(1)
memory usage: 529.4+ KB


In [8]:
ohlc.dtypes

Unnamed: 1    float64
Unnamed: 2    float64
Unnamed: 3    float64
Unnamed: 4    float64
Unnamed: 5      int64
dtype: object

In [9]:
ohlc.tail()

,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5
Date,,,,,
2025-09-29,254.429993,255.000000,253.009995,254.559998,40127700
2025-09-30,254.630005,255.919998,253.110001,254.860001,37704300
2025-10-01,255.449997,258.790009,254.929993,255.039993,48713900
2025-10-02,257.130005,258.179993,254.149994,256.579987,42630200
2025-10-03,258.019989,259.239990,253.949997,254.669998,49107000


In [10]:
numeric_cols = ['Open', 'High', 'Low', 'Close']

ohlc.columns = ['Open', 'High', 'Low', 'Close', 'Volume']

for col in numeric_cols:
    ohlc[col] = pd.to_numeric(ohlc[col], errors='coerce')


In [11]:
ohlc.describe(include='all')

,Open,High,Low,Close,Volume
count,11294.000000,11294.000000,11294.000000,11294.000000,1.129400e+04
mean,26.689600,26.958130,26.394175,26.666814,3.111597e+08
std,55.240040,55.781888,54.632914,55.179857,3.337334e+08
min,0.037681,0.038109,0.037681,0.038109,0.000000e+00
25%,0.245137,0.250706,0.240161,0.245127,1.076768e+08
50%,0.466639,0.474725,0.459261,0.466445,1.998388e+08
75%,20.927646,21.097477,20.743933,20.965368,3.910452e+08
max,258.103729,259.239990,256.718662,257.276679,7.421641e+09


In [12]:
ohlc.memory_usage()

Index     90352
Open      90352
High      90352
Low       90352
Close     90352
Volume    90352
dtype: int64

In [13]:
print(len(ohlc['Close'].unique()))

10673


In [14]:
ohlc['Volume'] = pd.to_numeric(ohlc['Volume'], errors='coerce')
ohlc['High'] = pd.to_numeric(ohlc['High'], errors='coerce')
ohlc['Low'] = pd.to_numeric(ohlc['Low'], errors='coerce')
ohlc['Open'] = pd.to_numeric(ohlc['Open'], errors='coerce')
ohlc['Close'] = pd.to_numeric(ohlc['Close'], errors='coerce')


all_time_low = ohlc['Low'].min()
all_time_low_date = ohlc['Low'].idxmin()
print(f'All time low: {all_time_low} on {all_time_low_date}')

all_time_high = ohlc['High'].max()
all_time_high_date = ohlc['High'].idxmax()
print(f'All time high: {all_time_high} on {all_time_high_date}')

all_time_low_vol = ohlc['Volume'].min()
all_time_low_vol_date = ohlc['Volume'].idxmin()
print(f'All time low volume: {all_time_low_vol} on {all_time_low_vol_date}')

all_time_high_vol = ohlc['Volume'].max()
all_time_high_vol_date = ohlc['Volume'].idxmax()
print(f'All time high volume: {all_time_high_vol} on {all_time_high_vol_date}')

All time low: 0.037681121379137 on 1982-07-08
All time high: 259.239990234375 on 2025-10-03
All time low volume: 0 on 1981-08-10
All time high volume: 7421640800 on 2000-09-29


Financials

In [15]:
balance_sheet.tail()

,2024-09-30,2023-09-30,2022-09-30,2021-09-30
Cash Cash Equivalents And Short Term Investments,6.517100e+10,6.155500e+10,4.830400e+10,6.263900e+10
Other Short Term Investments,3.522800e+10,3.159000e+10,2.465800e+10,2.769900e+10
Cash And Cash Equivalents,2.994300e+10,2.996500e+10,2.364600e+10,3.494000e+10
Cash Equivalents,2.744000e+09,1.606000e+09,5.100000e+09,1.763500e+10
Cash Financial,2.719900e+10,2.835900e+10,1.854600e+10,1.730500e+10


In [16]:
cashflow.tail()

,2024-09-30,2023-09-30,2022-09-30,2021-09-30
Deferred Tax,NaN,NaN,8.950000e+08,-4.774000e+09
Deferred Income Tax,NaN,NaN,8.950000e+08,-4.774000e+09
Depreciation Amortization Depletion,1.144500e+10,1.151900e+10,1.110400e+10,1.128400e+10
Depreciation And Amortization,1.144500e+10,1.151900e+10,1.110400e+10,1.128400e+10
Net Income From Continuing Operations,9.373600e+10,9.699500e+10,9.980300e+10,9.468000e+10


In [17]:
income_statement.tail()

,2024-09-30,2023-09-30,2022-09-30,2021-09-30
Selling General And Administration,2.609700e+10,2.493200e+10,2.509400e+10,2.197300e+10
Gross Profit,1.806830e+11,1.691480e+11,1.707820e+11,1.528360e+11
Cost Of Revenue,2.103520e+11,2.141370e+11,2.235460e+11,2.129810e+11
Total Revenue,3.910350e+11,3.832850e+11,3.943280e+11,3.658170e+11
Operating Revenue,3.910350e+11,3.832850e+11,3.943280e+11,3.658170e+11


Info

capital_gains, fast_info, history and splits are redunant

In [18]:
actions.tail()

,Dividends,Stock Splits
Date,,
2024-08-12 00:00:00-04:00,0.25,0.0
2024-11-08 00:00:00-05:00,0.25,0.0
2025-02-10 00:00:00-05:00,0.25,0.0
2025-05-12 00:00:00-04:00,0.26,0.0
2025-08-11 00:00:00-04:00,0.26,0.0


In [19]:
capital_gains.tail()

,0


In [20]:
info.tail() # Might be useful

,Unnamed: 0,0
177,exchangeDataDelayedBy,0
178,averageAnalystRating,2.0 - Buy
179,cryptoTradeable,False
180,displayName,Apple
181,trailingPegRatio,2.4662


In [21]:
for value in info.iloc[:, 0]:
    print(value, end=', ')

address1, city, state, zip, country, phone, website, industry, industryKey, industryDisp, sector, sectorKey, sectorDisp, longBusinessSummary, fullTimeEmployees, companyOfficers, auditRisk, boardRisk, compensationRisk, shareHolderRightsRisk, overallRisk, governanceEpochDate, compensationAsOfEpochDate, irWebsite, executiveTeam, maxAge, priceHint, previousClose, open, dayLow, dayHigh, regularMarketPreviousClose, regularMarketOpen, regularMarketDayLow, regularMarketDayHigh, dividendRate, dividendYield, exDividendDate, payoutRatio, fiveYearAvgDividendYield, beta, trailingPE, forwardPE, volume, regularMarketVolume, averageVolume, averageVolume10days, averageDailyVolume10Day, bid, ask, bidSize, askSize, marketCap, fiftyTwoWeekLow, fiftyTwoWeekHigh, allTimeHigh, allTimeLow, priceToSalesTrailing12Months, fiftyDayAverage, twoHundredDayAverage, trailingAnnualDividendRate, trailingAnnualDividendYield, currency, tradeable, enterpriseValue, profitMargins, floatShares, sharesOutstanding, sharesShort,

Analysis and holdings

In [22]:
earnings_estimate.tail()

,period,avg,low,high,yearAgoEps,numberOfAnalysts,growth
0,0q,1.75532,1.59,1.82,0.97000,31,0.8096
1,+1q,2.48185,2.28,2.67,2.40000,26,0.0341
2,0y,7.36557,7.09,7.47,6.08000,42,0.2114
3,+1y,7.99103,7.13,9.00,7.36557,42,0.0849


In [23]:
earnings_history.tail()

,quarter,epsActual,epsEstimate,epsDifference,surprisePercent
0,2024-09-30,0.97,0.94654,0.02,0.0248
1,2024-12-31,2.40,2.34102,0.06,0.0252
2,2025-03-31,1.65,1.62253,0.03,0.0169
3,2025-06-30,1.57,1.42572,0.14,0.1012


In [24]:
eps_revisions.tail()

,period,upLast7days,upLast30days,downLast30days,downLast7Days
0,0q,2,3,1,0
1,+1q,2,3,2,0
2,0y,2,4,2,0
3,+1y,2,5,2,0


In [25]:
eps_trend.tail()

,period,current,7daysAgo,30daysAgo,60daysAgo,90daysAgo
0,0q,1.75532,1.76050,1.75923,1.75709,1.65350
1,+1q,2.48185,2.49325,2.47123,2.46775,2.41761
2,0y,7.36557,7.36913,7.38088,7.37341,7.17122
3,+1y,7.99103,7.99959,7.96048,7.95049,7.81165


In [26]:
growth_estimates.tail()

,period,stockTrend,indexTrend
0,0q,0.0745,0.0689
1,+1q,0.0366,0.0607
2,0y,0.0960,0.0910
3,+1y,0.0835,0.1419
4,LTG,NaN,0.1220


In [27]:
insider_purchases.tail()

,Unnamed: 0,Insider Purchases Last 6m,Shares,Trans
2,2,Net Shares Purchased (Sold),-109611,7.0
3,3,Total Insider Shares Held,292355680,NaN
4,4,% Net Shares Purchased (Sold),0,NaN
5,5,% Buy Shares,0,NaN
6,6,% Sell Shares,0,NaN


In [28]:
insider_roster_holders.tail()

,Unnamed: 0,Name,Position,URL,Most Recent Transaction,Latest Transaction Date,Shares Owned Directly,Position Direct Date
5,5,O'BRIEN DEIRDRE,Officer,NaN,Sale,2025-08-08,136687,2025-08-08
6,6,PAREKH KEVAN,Chief Financial Officer,NaN,Sale,2025-04-23,4569,2025-04-23
7,7,SUGAR RONALD D,Director,NaN,Conversion of Exercise of derivative security,2025-01-31,109311,2025-01-31
8,8,WAGNER SUSAN L,Director,NaN,Conversion of Exercise of derivative security,2025-01-31,68533,2025-01-31
9,9,WILLIAMS JEFFREY E,Chief Operating Officer,NaN,Sale,2025-04-02,390059,2025-04-02


In [29]:
institutional_holders.tail()

,Unnamed: 0,Date Reported,Holder,pctHeld,Shares,Value,pctChange
5,5,2025-06-30,"Berkshire Hathaway, Inc",0.0189,280000000,72245596923,-0.0667
6,6,2025-06-30,Morgan Stanley,0.0157,233198646,60169912078,-0.0333
7,7,2025-06-30,JPMORGAN CHASE & CO,0.0145,214606399,55372740712,0.0798
8,8,2025-06-30,Price (T.Rowe) Associates Inc,0.0137,202720404,52305916412,-0.0558
9,9,2025-06-30,NORGES BANK,0.0128,189804820,48973437571,0.0141


In [30]:
major_holders.tail()

,Unnamed: 0,Value
0,insidersPercentHeld,0.01970
1,institutionsPercentHeld,0.63630
2,institutionsFloatPercentHeld,0.64909
3,institutionsCount,6949.00000


In [31]:
mutualfunds_holders.tail()

,Unnamed: 0,Date Reported,Holder,pctHeld,Shares,Value,pctChange
5,5,2025-06-30,VANGUARD INDEX FUNDS-Vanguard Growth Index Fund,0.0100,147872804,38154139263,0.0145
6,6,2025-08-31,"Invesco QQQ Trust, Series 1",0.0084,124259866,32061529260,-0.0035
7,7,2025-06-30,VANGUARD INSTITUTIONAL INDEX FUNDS-Vanguard In...,0.0062,91543381,23620022159,-0.0115
8,8,2025-05-31,VANGUARD WORLD FUND-Vanguard Information Techn...,0.0050,74018868,19098347508,0.0272
9,9,2025-08-31,iShares Trust-iShares Russell 1000 Growth ETF,0.0037,55486808,14316705590,-0.0010


In [32]:
reccomendations.tail()

,Unnamed: 0,period,strongBuy,buy,hold,sell,strongSell
0,0,0m,5,24,14,2,3
1,1,-1m,5,23,15,1,3
2,2,-2m,5,22,15,1,1
3,3,-3m,5,23,15,1,1


In [33]:
revenue_estimate.tail()

,period,avg,low,high,numberOfAnalysts,yearAgoRevenue,growth
0,0q,101719746060,97854000000,103220000000,28,94930000000,0.0715
1,+1q,130743862550,125551070150,138910000000,22,124300000000,0.0518
2,0y,415273963930,404529000000,431783000000,41,391035000000,0.0620
3,+1y,438145978760,408100000000,477463000000,43,415273963930,0.0551


In [34]:
sustainability.tail()

,Unnamed: 0,esgScores
31,nuclear,False
32,pesticides,False
33,palmOil,False
34,coal,False
35,tobacco,False


In [35]:
upgrades_downgrades.tail()

,GradeDate,Firm,ToGrade,FromGrade,Action,priceTargetAction,currentPriceTarget,priorPriceTarget
953,2016-09-22 10:22:56,Nomura,Buy,NaN,main,Raises,135.0,120.0
954,2016-09-16 11:12:49,CLSA,Buy,NaN,main,Raises,32.5,30.0
955,2016-09-15 12:10:08,Credit Suisse,Outperform,NaN,main,Maintains,150.0,0.0
956,2016-09-15 12:08:11,BTIG,Buy,NaN,main,Raises,133.0,124.0
957,2012-09-12 11:51:00,Hudson Square,Buy,NaN,main,Raises,900.0,700.0


VIX

In [36]:
vix.tail()

,0,1,2,3,4,5
9004,2025-10-02,16.629999,16.920000,15.930000,16.120001,0
9005,2025-10-03,16.650000,17.059999,16.200001,16.350000,0
9006,2025-10-06,16.370001,17.090000,16.290001,16.740000,0
9007,2025-10-07,17.240000,17.549999,16.190001,16.580000,0
9008,2025-10-08,16.559999,17.200001,16.459999,17.170000,0


# Data preprocessing

In [37]:
data = ohlc.copy()

data.index = pd.to_datetime(data.index, format='%Y-%m-%d')

# Feature engineering

Adding percentage change of the closing price

In [38]:
data['Return'] = data['Close'].pct_change()

Adding volatility indicators

In [39]:
data['Volatility_20'] = data['Close'].rolling(window=20).std()
data['Volatility_50'] = data['Close'].rolling(window=50).std()

Adding ATR (Average True Range) — captures volatility from high/low ranges

In [40]:
high_low = data['High'] - data['Low']
high_close = (data['High'] - data['Close'].shift()).abs()
low_close = (data['Low'] - data['Close'].shift()).abs()

data['TR'] = high_low.combine(high_close, max).combine(low_close, max)
data['ATR_14'] = data['TR'].rolling(window=14).mean()

Adding moving averages

In [41]:
windows = [5, 20, 50, 200]

for w in windows:
    data[f'SMA_{w}'] = data['Close'].rolling(window=w).mean()
    data[f'EMA_{w}'] = data['Close'].ewm(span=w, adjust=False).mean()

Momentum indicators

In [42]:
# RSI (Relative Strength Index)
delta = data['Close'].diff()
gain = delta.clip(lower=0)
loss = -delta.clip(upper=0)

avg_gain = gain.rolling(window=14).mean()
avg_loss = loss.rolling(window=14).mean()

rs = avg_gain / avg_loss
data['RSI_14'] = 100 - (100 / (1 + rs))

# MACD (Moving Average Convergence Divergence)
ema_12 = data['Close'].ewm(span=12, adjust=False).mean()
ema_26 = data['Close'].ewm(span=26, adjust=False).mean()

data['MACD'] = ema_12 - ema_26
data['MACD_signal'] = data['MACD'].ewm(span=9, adjust=False).mean()

# Stochastic Oscillator
low_14 = data['Low'].rolling(14).min()
high_14 = data['High'].rolling(14).max()

data['Stochastic'] = 100 * (data['Close'] - low_14) / (high_14 - low_14)

# ADX (Average Directional Index)
high = data['High']
low = data['Low']
close = data['Close']

plus_dm = high.diff()
minus_dm = low.diff().abs()
plus_dm[plus_dm < 0] = 0
minus_dm[minus_dm < 0] = 0

tr = pd.concat([high - low, (high - close.shift()).abs(), (low - close.shift()).abs()], axis=1).max(axis=1)
atr = tr.rolling(14).mean()

plus_di = 100 * (plus_dm.rolling(14).sum() / atr)
minus_di = 100 * (minus_dm.rolling(14).sum() / atr)

dx = 100 * (abs(plus_di - minus_di) / (plus_di + minus_di))
data['ADX_14'] = dx.rolling(14).mean()

# On-Balance Volume (OBV)
data['OBV'] = (np.sign(data['Close'].diff()) * data['Volume']).fillna(0).cumsum()

# Money Flow Index (MFI) - fully vectorized
typical_price = (data['High'] + data['Low'] + data['Close']) / 3
money_flow = typical_price * data['Volume']
tp_diff = typical_price.diff()

positive_mf = money_flow.where(tp_diff > 0, 0).rolling(14).sum()
negative_mf = money_flow.where(tp_diff < 0, 0).rolling(14).sum()
data['MFI_14'] = 100 * (positive_mf / (positive_mf + negative_mf))

# Basic Candlestick Patterns
data['Doji'] = abs(data['Close'] - data['Open']) <= 0.1 * (data['High'] - data['Low'])
data['Hammer'] = ((data['High'] - data['Low']) > 3 * (data['Open'] - data['Close'])) & \
                 ((data['Close'] - data['Low']) / (0.001 + data['High'] - data['Low']) > 0.6)
data['Engulfing'] = ((data['Close'] > data['Open'].shift()) &
                     (data['Open'] < data['Close'].shift()) &
                     (data['Close'] - data['Open'] > data['Open'].shift() - data['Close'].shift()))

Fundamental features

Dividends (0/1)

In [43]:
# Make sure data index is timezone-naive
data.index = pd.to_datetime(data.index).tz_localize(None)

# Convert dividends index to datetime (force conversion and make tz-naive safely)
dividend_dates = pd.to_datetime(dividends.index, utc=True, errors='raise')  # convert all to UTC first
dividend_dates = dividend_dates.tz_convert(None)  # then drop tz info

# Remove any invalid (NaT) entries
dividend_dates = dividend_dates.dropna()

# Compare only by date (normalize removes the time part)
dividend_dates = set(dividend_dates.normalize())

# Create Dividends (0/1) column in data
data['Dividends'] = data.index.normalize().isin(dividend_dates).astype(int)

Split (0/1)

In [44]:
splits_dates = pd.to_datetime(splits.index, utc=True, errors='raise')
splits_dates = splits_dates.tz_convert(None)

splits_dates = splits_dates.dropna()

splits_dates = set(splits_dates.normalize())

data['Splits'] = data.index.normalize().isin(splits_dates).astype(int)

VIX

The VIX (Volatility Index) is often called the “fear gauge” of the market. It represents the market’s expectation of 30-day forward-looking volatility in the S&P 500 index, derived from options prices.
* High VIX → Market expects high volatility, often during market stress or uncertainty.
* Low VIX → Market expects low volatility, generally in calm or bullish periods.
* VIX is expressed in percentage points (e.g., VIX = 20 means the market expects ~20% annualized volatility over the next 30 days).

Important: VIX is not a price index; it’s a measure of expected volatility.

In [45]:
vix.columns = ['Date', 'Close', 'High', 'Low', 'Open', 'Volume']
vix['Date'] = pd.to_datetime(vix['Date'])
vix.set_index('Date', inplace=True)

vix.drop(columns=['Volume'], inplace=True)

data = data.join(vix, how='left', rsuffix='_VIX')

Date
2025-09-29    16.120001
2025-09-30    16.280001
2025-10-01    16.290001
2025-10-02    16.629999
2025-10-03    16.650000
Name: Close_VIX, dtype: float64

In [49]:
# VIX returns
data['VIX_Return'] = data['Close_VIX'].pct_change()

# Volatility
data['VIX_Volatility_20'] = data['Close_VIX'].rolling(window=20).std()
data['VIX_Volatility_50'] = data['Close_VIX'].rolling(window=50).std()

# Trend
data['VIX_SMA_20'] = data['Close_VIX'].rolling(window=20).mean()
data['VIX_EMA_20'] = data['Close_VIX'].ewm(span=20, adjust=False).mean()
data['VIX_RollingVol'] = data['Close_VIX'].rolling(5).std()
data['VIX_Change'] = data['Close_VIX'] - data['Close_VIX'].shift(1)

# Lagged features
for lag in range(1, 6):
    data[f'VIX_Close_lag{lag}'] = data['Close_VIX'].shift(lag)